In [2]:
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')


# Other imports and helper functions
import time
import itertools
import numpy as np

# Graphics and plotting.
print('Installing mediapy:')
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()
import mujoco
import mujoco.viewer

In [30]:
xml = """
<mujoco>
<worldbody>
 <!-- Bottom of the box -->
        <body name="bottom" pos="0 0 -0.5">
            <geom type="box" size=".1 .1 .05" rgba="0.8 0.6 0.3 1" />
        </body>

        <!-- Front wall -->
        <body name="front_wall" pos="0 0 -0.25">
            <geom type="box" size=".1 .05 .01" rgba="0.8 0.6 0.3 1" />
        </body>

        <!-- Back wall -->
        <body name="back_wall" pos="0 0 0.25">
            <geom type="box" size=".1 .05 .01" rgba="0.8 0.6 0.3 1" />
        </body>

        <!-- Left wall -->
        <body name="left_wall" pos="-0.5 0 0">
            <geom type="box" size=".05 .1 .01" rgba="0.8 0.6 0.3 1" />
        </body>

        <!-- Right wall -->
        <body name="right_wall" pos="0.5 0 0">
            <geom type="box" size=".05 .01 .04" rgba="0.8 0.6 0.3 1" />
        </body>
  <composite type="particle" count="10 10 10" spacing="0.07" offset="0 0 1">
    <geom size=".02" rgba=".8 .2 .1 1"/>
  </composite>
</worldbody>
</mujoco>
"""

In [31]:
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

with mujoco.viewer.launch_passive(model, data) as viewer:
  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()
  while viewer.is_running() and time.time() - start < 30:
    step_start = time.time()

    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(model, data)

    # Example modification of a viewer option: toggle contact points every two seconds.
    with viewer.lock():
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(data.time % 2)

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = model.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)